# Cell Below is CSV Combiner 

Run the cell below using the triangle button on the top left or clicking in the cell and pressing Shift + Enter to run it. The code will use the files names from the "data" folder. It will assume protein names from each file names by using everything before the first underscore (_) then will combine into one csv file with columns as different proteins from each file. 

Note that there will be a preview of the first 10 rows of what the final csv looks like so you can check it before moving to save it in the following cell. 

In [5]:
import pandas as pd
from IPython.display import FileLink
import os

def merge_protein_csvs(folder_path):
    # Step 1: Get all CSVs
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Step 2: Identify the DAPI file (case-insensitive search)
    dapi_file = next((f for f in csv_files if 'dapi' in f.lower()), None)
    if not dapi_file:
        raise ValueError("No DAPI file found in folder.")

    # Step 3: Load DAPI DataFrame as base
    dapi_df = pd.read_csv(os.path.join(folder_path, dapi_file), skiprows=3)
    dapi_df.columns = [col.strip().rstrip(',').upper() for col in dapi_df.columns]
    dapi_df = dapi_df[['POSITION X', 'POSITION Y']].drop_duplicates()
    
    # Ensure we don't modify DAPI filename in the loop
    remaining_files = [f for f in csv_files if f != dapi_file]

    # Step 4: Process each protein file
    for file in remaining_files:
        filepath = os.path.join(folder_path, file)
        df = pd.read_csv(filepath, skiprows=3)
        df.columns = [col.strip().rstrip(',').upper() for col in df.columns]
        
        # Drop duplicates just in case
        df = df[['POSITION X', 'POSITION Y']].drop_duplicates()
        
        # Extract protein name and convert to uppercase
        protein_name = file.split('_')[0].upper()
        
        # Mark presence of (X,Y) in current protein file
        df[protein_name] = 1
        
        # Merge into base (DAPI) on POSITION X and Y
        dapi_df = dapi_df.merge(df, on=['POSITION X', 'POSITION Y'], how='left')
    
    # Step 5: Replace NaN with 0 in presence columns
    presence_cols = [col for col in dapi_df.columns if col not in ['POSITION X', 'POSITION Y']]
    dapi_df[presence_cols] = dapi_df[presence_cols].fillna(0).astype(int)

    return dapi_df


folder_path = './data/'
combined_csv = merge_protein_csvs(folder_path)
combined_csv


,POSITION X,POSITION Y,CD3,CD4,FOXP3,NK1.1
0,55878.777,41778.797,0,1,0,0
1,57359.824,40297.508,0,1,0,0
2,56290.895,41362.340,0,1,0,0
3,56432.293,41224.496,0,1,0,0
4,55830.648,41824.309,0,1,0,0
...,...,...,...,...,...,...
114264,53369.691,38341.012,0,1,0,0
114265,53274.793,38437.641,0,0,0,0
114266,53579.414,38130.742,0,1,0,0
114267,53456.289,38254.949,0,1,0,0


# Use cell below to save the combined csv file after ensuring the preview looks correct above. 

Run the cell below by clicking the triangle in the top left or clicking in the cell and pressing shift + enter. Make sure to renmae the file name to whatever you need in the part to left of the green arrow make sure to leave the name in the quotes. 

The combined csv file will be saved in a folder called "combined_csvs" there will also be a hyperlink below that will open up the csv file to make sure you can see where it is stored and view the full file. 

In [7]:
output_dir = "combined_csvs"
os.makedirs(output_dir, exist_ok=True)  

file_name = "final_output.csv" # <----------- Change the file name here (leave the .csv extension)
output_path = os.path.join(output_dir, file_name)

combined_csv.to_csv(output_path, index=False)

FileLink(output_path)


/Users/patrick/Desktop2/lab_proj1/combined_csvs/final_output.csv